# OPTICS Clustering with H3 Hexagonal Aggregation and Heterogeneity Metrics

This notebook demonstrates:
- Generating synthetic spatial point data
- Running **OPTICS** clustering and visualizing clusters
- Adding a **reachability plot** for OPTICS
- Comparing with **DBSCAN** clustering
- Aggregating points into **H3 hexagons** for density analysis
- Visualizing clusters and density overlay
- Computing heterogeneity metrics (variance, Gini coefficient, entropy)
- Providing next steps for spatial analysis

**Dependencies:** `geopandas`, `scikit-learn`, `h3`, `matplotlib`, `numpy`, `pandas`


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from sklearn.cluster import OPTICS, DBSCAN
from sklearn.datasets import make_blobs
import h3


## Generate Synthetic Spatial Data

In [ ]:
# Generate synthetic spatial data using make_blobs
X, _ = make_blobs(n_samples=500, centers=[[0,0],[5,5],[10,1]], cluster_std=[0.5, 0.8, 0.6], random_state=42)

# Convert to GeoDataFrame
geometry = [Point(xy) for xy in X]
gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:3857")  # Projected CRS for clustering
print(gdf.head())


## Run OPTICS Clustering and Visualize Clusters

In [ ]:
# Run OPTICS clustering
optics = OPTICS(min_samples=10, xi=0.05, min_cluster_size=0.05)
optics.fit(X)

# Add cluster labels to GeoDataFrame
gdf['optics_cluster'] = optics.labels_

# Visualize OPTICS clusters
fig, ax = plt.subplots(figsize=(8,6))
gdf.plot(column='optics_cluster', categorical=True, legend=True, ax=ax, cmap='tab20', markersize=30)
plt.title("OPTICS Clustering")
plt.show()


## Reachability Plot for OPTICS

In [ ]:
# Reachability plot for OPTICS
plt.figure(figsize=(10,4))
plt.plot(optics.reachability_[optics.ordering_])
plt.title("Reachability Plot (OPTICS)")
plt.xlabel("Points (ordered)")
plt.ylabel("Reachability Distance")
plt.show()


## Compare with DBSCAN Clustering

In [ ]:
# Run DBSCAN clustering for comparison
dbscan = DBSCAN(eps=1.0, min_samples=10)
dbscan.fit(X)

# Add DBSCAN cluster labels
gdf['dbscan_cluster'] = dbscan.labels_

# Visualize DBSCAN clusters
fig, ax = plt.subplots(figsize=(8,6))
gdf.plot(column='dbscan_cluster', categorical=True, legend=True, ax=ax, cmap='tab20', markersize=30)
plt.title("DBSCAN Clustering")
plt.show()


## Aggregate Points into H3 Hexagons

In [ ]:
# Convert coordinates to lat/lon for H3 (simulate by treating X as lat/lon for demonstration)
# In real case, reproject to EPSG:4326
latlon_coords = [(float(y), float(x)) for x,y in X]

# Choose H3 resolution
resolution = 8

# Assign each point to an H3 hexagon
h3_indices = [h3.geo_to_h3(lat, lon, resolution) for lat, lon in latlon_coords]

# Aggregate counts per hexagon
hex_counts = pd.Series(h3_indices).value_counts().reset_index()
hex_counts.columns = ['h3_index', 'count']

# Get hexagon boundaries
hex_polygons = [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in hex_counts['h3_index']]

# Create GeoDataFrame of hexagons
hex_gdf = gpd.GeoDataFrame(hex_counts, geometry=hex_polygons, crs="EPSG:4326")
print(hex_gdf.head())


## Visualize Clusters and Density Overlay

In [ ]:
# Visualize H3 density overlay and OPTICS clusters
fig, ax = plt.subplots(figsize=(10,8))
hex_gdf.plot(column='count', cmap='OrRd', legend=True, ax=ax, alpha=0.6)
gdf.plot(column='optics_cluster', categorical=True, ax=ax, markersize=10, cmap='tab20', legend=True)
plt.title("OPTICS Clusters with H3 Density Overlay")
plt.show()


## Compute Heterogeneity Metrics

In [ ]:
# Compute heterogeneity metrics for hexagon counts
counts = hex_gdf['count'].values

# Variance
variance = np.var(counts)

# Gini coefficient
def gini(array):
    array = np.sort(array)
    n = len(array)
    cumulative = np.cumsum(array)
    return (n + 1 - 2 * np.sum(cumulative) / cumulative[-1]) / n

gini_coeff = gini(counts)

# Entropy
prob = counts / counts.sum()
entropy = -np.sum(prob * np.log(prob))

print(f"Variance: {variance}")
print(f"Gini Coefficient: {gini_coeff}")
print(f"Entropy: {entropy}")


## Next Steps
- Apply workflow to real spatial data (ensure projected CRS for clustering).
- Experiment with different H3 resolutions for scale sensitivity.
- Use Local Moran's I or Gi* for hotspot analysis.
- Validate clustering results with Monte Carlo simulations.
